<center><img style='border-radius: 10%'  src="https://cdn.discordapp.com/attachments/749911555780378688/801538985808625714/rock-paper-scissors-pop-art-vector-design.jpg" width="600"><center>

<center><H1 Style =  'font-size: 300%'> Rock, paper, scissors</H1><center>
Onze submissie voor de rock, paper, scissors competitie. In deze competitie is het de bedoeling dat de tegenstander wordt verslagen door de agent zo goed mogelijk te trainen.<br>
Dit doen we door middel van een sequentieel neuraal netwerk.<br>
<br>
Door Ben van Kampen & Pim Scheele



In [ ]:
!pip install -q -U kaggle_environments

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Model maken & trainen<center><h2>

Doel van deze cel:
* Het trainen van een sequential model

Dit doen we door middel van de Keras API van Tensorflow

In [ ]:
# Imports
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, LSTM

EPOCH_NP = 100
INPUT_SHAPE = (1, -1, 1)
OUTPUT_SHAPE = (1, -1, 3)
DATA_FILE = "../input/rockpaperscissortrain/data.txt"
MODEL_FILE = "RPS_model.h5"

def simple_model():
    # Maak een simpel model
    new_model = Sequential()
    new_model.add(LSTM(units=64, input_dim=1, return_sequences=True, activation='sigmoid'))
    new_model.add(LSTM(units=64, return_sequences=True, activation='sigmoid'))
    new_model.add(LSTM(units=64, return_sequences=True, activation='sigmoid'))
    new_model.add(Dense(64, activation='relu'))
    new_model.add(Dense(64, activation='relu'))
    new_model.add(Dense(3, activation='softmax'))
    new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'categorical_crossentropy'])
    return new_model

def batch_generator(filename):
    # Genereer leer-data van een bestand
    with open(DATA_FILE, 'r') as data_file:
        for line in data_file:
            # Converteer lijn naar een numpy array
            data_vector = np.array(list(line[:-1]))
            # Voeg extra rijen toe (Dit heeft het model nodig)
            input_data = data_vector[np.newaxis, :-1, np.newaxis]
            temp = np_utils.to_categorical(data_vector, num_classes=3)
            # Voeg nog een rij toe voor missende data
            output_data = temp[np.newaxis, 1:]
            yield (input_data, output_data)

# Maak model aan
np.random.seed(7)
model = simple_model()

# Laad data en leer (Errors horen erbij, er kan maar een van de drie cellen gevuld worden. Omdat we werken met een try kan de except niet uitgecomment worden.)
for (input_data, output_data) in batch_generator(DATA_FILE):
    try:
        model.fit(input_data, output_data, epochs=100, batch_size=100)
    except:
        print("error")

# Evalueer het model
print("evaluating")
validation = '100101000110221110101002201101101101002201011012222210221011011101011122110010101010101'
input_validation = np.array(list(validation[:-1])).reshape(INPUT_SHAPE).astype(int)
output_validation = np_utils.to_categorical(np.array(list(validation[1:]))).reshape(OUTPUT_SHAPE).astype(int)
loss_and_metrics = model.evaluate(input_validation, output_validation, batch_size=6)

print("\n Evaluation results")

for i in range(len(loss_and_metrics)):
    print(model.metrics_names[i], loss_and_metrics[i])

model.save(MODEL_FILE)
del model

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Model inzetten tegen andere AI's<center><h2>

Doel van deze cel:
* Het model laten spelen tegen andere AI's
* Data vergaren van deze wedstrijden



In [ ]:
%%writefile modelrunner.py


# Imports
import matplotlib.pyplot as plt
import numpy as np
import random
import keras

INPUT_SHAPE = (1, -1, 1)
opponent_actions = []
agent_actions = []
model = keras.models.load_model('../input/rockpaperscissortrain/RPS_model.h5')


def agent(observation, configuration):
    # Eerste stap
    if observation.step == 0:
        action = random.randint(0,2)
        agent_actions.append(action)
        return action
    
    # Stappen 2 tot 100 als training data gebruiken
    if 0 < observation.step < 99:
        opponent_actions.append(observation.lastOpponentAction)
        action = random.randint(0,2)
        agent_actions.append(action)
        return action
    
    # Maak met het model, op basis van de laatste 99 stappen een predictie
    if observation.step >= 99:
        opponent_actions.append(observation.lastOpponentAction)
        input_test = np.array(opponent_actions[-99:]).reshape(INPUT_SHAPE)
        res = model.predict(input_test)
        prediction = np.argmax(res[0], axis=1)
        action = int(prediction[-1])
        agent_actions.append(action)
        return action

<h1 style='background:#fff13e; border:0; color:#4260f5'><center>Tegenstanders<center><h1>


<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Random Forest<center><h2>

Doel van de agent
* Met het random forest algoritme een voorspelling maken

In [ ]:
%%writefile random_forest_random.py

# Imports
import random
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

actions =  np.empty((0,0), dtype = int)
observations =  np.empty((0,0), dtype = int)
total_reward = 0

def random_forest_random(observation, configuration):
    global actions, observations, total_reward
    
    if observation.step == 0:
        action = random.randint(0,2)
        actions = np.append(actions , [action])
        return action
    
    if observation.step == 1:
        action = random.randint(0,2)
        actions = np.append(actions , [action])
        observations = np.append(observations , [observation.lastOpponentAction])
        # Houd de score bij
        winner = int((3 + actions[-1] - observation.lastOpponentAction) % 3);
        if winner == 1:
            total_reward = total_reward + 1
        elif winner == 2:
            total_reward = total_reward - 1        
        return action

    # Trek de tabel recht
    observations = np.append(observations , [observation.lastOpponentAction])
    
    # Bereid data voor op de training
    X_train = np.vstack((actions[:-1], observations[:-1])).T 
    shifted_observations = np.roll(observations, -1)
    y_train = shifted_observations[:-1].T
    
    # Bepaal op hoeveel data getrained wordt
    if len(X_train) > 25:
        random_window_size = 10 + random.randint(0,10)
        X_train = X_train[-random_window_size:]
        y_train = y_train[-random_window_size:]
   
    # Train het RF model
    model = RandomForestClassifier(n_estimators=25)
    model.fit(X_train, y_train)

    # Maak een voorspelling met het RF model
    X_test = np.empty((0,0), dtype = int)
    X_test = np.append(X_test, [int(actions[-1]), observation.lastOpponentAction])
    prediction = model.predict(X_test.reshape(1, -1))

    # Hou de score bij
    winner = int((3 + actions[-1] - observation.lastOpponentAction) % 3);
    if winner == 1:
        total_reward = total_reward + 1
    elif winner == 2:
        total_reward = total_reward - 1
   
    # Bereid actie voor
    action = int((prediction + 1) % 3)
    
    # Breek het patroon en begin random te spelen als het model verliest
    if total_reward < -2:
        win_tie = random.randint(0,1)
        action = int((prediction + win_tie) % 3)

    # Update de acties
    actions = np.append(actions , [action])

    # Actie 
    return action 

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Steen<center><h2>

Doel van de agent
* Altijd steen kiezen

In [ ]:
%%writefile rock.py

def rock(obs, config):
    return 0


<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Papier<center><h2>

Doel van de agent
* Altijd papier kiezen

In [ ]:
%%writefile paper.py

def paper(obs, config):
    return 1

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Schaar<center><h2>

Doel van de agent
* Altijd schaar kiezen

In [ ]:
%%writefile scissors.py

def scissors(obs, config):
    return 2

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Kopiëer Tegenstander<center><h2>

Doel van de agent:
* Kopieert de laatste stap van de tegenstander
* Als er geen laatste stap is, kies een willekeurige stap

In [ ]:
%%writefile copy_last_action.py
import random

def copy_opponent_agent(obs, config):
    # Kies laatste stap van de tegenstander, als er nog geens stappen zijn kies random
    if obs.step > 0:
        return obs.lastOpponentAction
    else:
        random.randint(0,2)

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Willekeurige stap<center><h2>

Doel van de agent:
* Doe een willekeurige stap

In [ ]:
%%writefile random_agent.py
import random

def copy_opponent_agent(obs, config):
        return random.randint(0,2)

<h2 style='background:#fff13e; border:0; color:#4260f5'><center>Drie keer hetzelfde<center><h2>

Doel van de agent:
* Kiest een willekeurige stap en herhaalt deze 3 keer

In [ ]:
%%writefile threetimesthecharm.py
import random

history = {
    "opponent":   [0,1],
    "action": [0,1,2],
}

# Check of de laatste drie zetten hetzelfde zijn, als dit zo is kies een random zet, zo niet, speel de laatste zet.
def threetimesthecharm(obs, config):
    last = history['action'][-1]
    secondlast = history['action'][-2]
    thirdlast = history['action'][-3]
    if last == secondlast and secondlast == thirdlast:
        randint = random.randint(0,2)
        history['action'].append(randint)
        return randint
    else:
        action = last
        history['action'].append(action)
        return action

<h1 style='background:#fff13e; border:0; color:#4260f5'><center>Testomgeving<center>

Maken van environment genaamd "rps" (Rock, paper, scissors). De environment doet 1000 stappen.

* kiezen welke files het tegen elkaar op gaan nemen.
* kiezen hoe het gevisualiseerd wordt.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate
env = make("rps", configuration={"episodeSteps : 1000"})
env.run(["rock.py", "modelrunner.py"])
env.render(mode="ipython", width=500, height=450)